In [81]:
import glob
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
from sklearn.mixture import GaussianMixture
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.io import imread, imshow, imsave
from skimage.color import rgb2gray
from skimage.exposure import histogram, equalize_hist
from skimage.filters import threshold_otsu, threshold_niblack

### Definindo os metodos de segmentação

#### k-means

In [82]:
def segmentation(img, name_cluster, k):
    
    cluster = None
    
    if name_cluster == "K-means":
        cluster = KMeans(n_clusters=k)
    
    elif name_cluster == "Agllomerative":
        cluster = AgglomerativeClustering(n_clusters=k)
    
    elif name_cluster == "Gaussian":
        cluster = GaussianMixture(n_components=k)
        
    elif name_cluster == "Birch":
        cluster = Birch(threshold=0.03, n_clusters=k)
        
  

    
    img_train = img.reshape(-1, 1)
    result = cluster.fit(img_train)
    
    labels = result.labels_
    cluster_centers = result.cluster_centers_
    
    cluster_centers = cluster_centers.reshape(1, -1)
    cluster_centers = list(cluster_centers[0])
    
    index_min = cluster_centers.index(min(cluster_centers))
    index_max = cluster_centers.index(max(cluster_centers))
    
    labels = ((labels != index_min) & (labels != index_max)).astype(np.uint8) * 255
    img_final = labels.reshape(img.shape)
    
    return img_final

<p> A escolha dos grupos para a segmentação é feita obtendo o index do menor centroide e fazendo uma binarização removendo os valores que pertencem ao grupo do mesmo. (Foi escolhido o menor centroide a partir de teste que indicavão que as áreas mais escuras eram determinados por esse centroide) <p>

#### Niblack e Otsu

In [83]:
def niblack(image, window_size, k):
    threshold = threshold_niblack(image, window_size, k)
    binary_image = image > threshold
    
    return binary_image.astype(np.uint8) * 255

In [84]:
def otsu(image):
    threshold_value = threshold_otsu(image)
    binary_image = image > threshold_value
    
    return binary_image.astype(np.uint8) * 255

In [85]:
def kmeans_niblack(img, k1, window_size, k2):
    kmeans = KMeans(n_clusters=k1)
    img_train = img.reshape(-1, 1)
    result = kmeans.fit(img_train)
    
    labels = result.labels_
    new_img = labels.reshape(img.shape)
    img_final = niblack(new_img, window_size, k2)
    
    return img_final

In [86]:
def kmeans_otsu(img, k):
    kmeans = KMeans(n_clusters=k)
    img_train = img.reshape(-1, 1)
    result = kmeans.fit(img_train)
    
    labels = result.labels_
    new_img = labels.reshape(img.shape)
    img_final = otsu(new_img)
    
    return img_final

#### Outro algoritmo de agrupamento

## Métricas

#### Intersection over Union (IoU)

In [87]:
def iou(pred, mask):
    # Calcular a interseção
    inter = (pred & mask).sum()

    # Calcular a união
    union = (pred | mask).sum()

    # Calcular e retornar o IoU
    return inter / union

#### Dice Coefficient

In [88]:
def dice(pred, mask):
    # Calcular a interseção
    inter = (pred & mask).sum()
    
    # Calcular o tatal de pixels das macaras
    total = pred.sum() + mask.sum()
    
    return (2 * inter) / total

## Carregando o conjunto de dados

In [89]:
path_img = "ultrassound/cropped/*.bmp"
path_mask = "ultrassound/masks/*.bmp"

images_dir = glob.glob(path_img)
masks_dir = glob.glob(path_mask)

images = [imread(img) for img in images_dir]
masks = [imread(mask) for mask in masks_dir]

In [90]:
images_gray = [rgb2gray(img) for img in images]

#### Função para salvar as bases

In [91]:
def save_base(images, seg_type, path, images_dir):
    if seg_type == "kmeans":
        for i in range(len(images)):
            image_seg = segmentation(images[i], "K-means", 4)
            img_name = images_dir[i][20:]
            imsave(path+img_name, image_seg)
    
    elif seg_type == "niblack":
        for i in range(len(images)):
            image_seg = kmeans_niblack(images[i], 4, 15, 0.8)
            img_name = images_dir[i][20:]
            imsave(path+img_name, image_seg)
    
    elif seg_type == "otsu":
        for i in range(len(images)):
            image_seg = kmeans_otsu(images[i], 4)
            img_name = images_dir[i][20:]
            imsave(path+img_name, image_seg)
    
    return 1

#### Criando a base com a segmentação pelo k-means

##### K-means

In [92]:
save_base(images_gray, "kmeans", "kmeans/", images_dir)

/home/killeglass/anaconda3/envs/face/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/tmp/ipykernel_16585/580078482.py:6: UserWarning: kmeans/1100208028.bmp is a low contrast image
  imsave(path+img_name, image_seg)
/home/killeglass/anaconda3/envs/face/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/tmp/ipykernel_16585/580078482.py:6: UserWarning: kmeans/1100209013.bmp is a low contrast image
  imsave(path+img_name, image_seg)
/home/killeglass/anaconda3/envs/face/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to su

1

##### niblack

In [93]:
save_base(images_gray, "niblack", "niblack/", images_dir)

/home/killeglass/anaconda3/envs/face/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/killeglass/anaconda3/envs/face/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/killeglass/anaconda3/envs/face/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/killeglass/anaconda3/envs/face/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppre

1

##### otsu

In [94]:
save_base(images_gray, "otsu", "otsu/", images_dir)

/home/killeglass/anaconda3/envs/face/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/killeglass/anaconda3/envs/face/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/killeglass/anaconda3/envs/face/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/killeglass/anaconda3/envs/face/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppre

1

### Avaliação

In [95]:
def avaliacao(pred, mask, metric):
    num_imgs = len(pred)
    somatorio = 1
    if metric == "iou":
        for i in range(num_imgs):
            somatorio += iou(pred[i], mask[i])
    
    elif metric == "dice":
        for i in range(num_imgs):
            somatorio += dice(pred[i], mask[i])
    
    return somatorio / num_imgs

In [96]:
path_kmeans = "kmeans/*.bmp"
path_niblack = "niblack/*.bmp"
path_otsu = "otsu/*.bmp"

kmeans_dir = glob.glob(path_kmeans)
niblack_dir = glob.glob(path_niblack)
otsu_dir = glob.glob(path_otsu)

images_kmeans = [imread(img) for img in kmeans_dir]
images_niblack = [imread(img) for img in niblack_dir]
images_otsu = [imread(img) for img in otsu_dir]

In [97]:
result_iou_kmeans = avaliacao(images_kmeans, masks, "iou")
result_dice_kmeans = avaliacao(images_kmeans, masks, "dice")

In [98]:
result_iou_niblack = avaliacao(images_niblack, masks, "iou")
result_dice_niblack = avaliacao(images_niblack, masks, "dice")

In [99]:
result_iou_otsu = avaliacao(images_otsu, masks, "iou")
result_dice_otsu = avaliacao(images_otsu, masks, "dice")

#### Resultados

In [100]:
print("Kmeans\nIoU: {}\nDice: {}\n".format(result_iou_kmeans, result_dice_kmeans))

Kmeans
IoU: 0.007228968494624213
Dice: 0.009591214767861338



In [101]:
print("Niblack\nIoU: {}\nDice: {}\n".format(result_iou_niblack, result_dice_niblack))

Niblack
IoU: 0.27032348320142796
Dice: 0.41345242600108995



In [102]:
print("Otsu\nIoU: {}\nDice: {}\n".format(result_iou_otsu, result_dice_otsu))

Otsu
IoU: 0.2015754583541163
Dice: 0.3187677834540017

